In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
train = pd.read_csv('Train_aBjfeNk.csv')
test = pd.read_csv('Test_LqhgPWU.csv')


In [7]:
label = train['Segmentation']
train = train.drop(['Segmentation'], axis=1)

In [8]:
print(train.shape)
print(test.shape)

(8068, 10)
(2627, 10)


In [9]:
train.nunique()

ID                 8068
Gender                2
Ever_Married          2
Age                  67
Graduated             2
Profession            9
Work_Experience      15
Spending_Score        3
Family_Size           9
Var_1                 7
dtype: int64

In [11]:
train.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
dtype: int64

In [12]:
train.Var_1.fillna('Cat_10', inplace=True)
train.Var_1 = train['Var_1'].apply(lambda x:int(str(x).split('_')[1]))

test.Var_1.fillna('Cat_10', inplace=True)
test.Var_1 = test['Var_1'].apply(lambda x:int(str(x).split('_')[1]))

In [13]:
Profession = {
    'Healthcare':0,
    'Engineer':1,
    'Lawyer':2,
    'Entertainment':3,
    'Artist':4,
    'Executive':5,
    'Doctor':6,
    'Homemaker':7,
    'Marketing':8,
    np.nan:10
}
Gender = {
    'Male':0,
    'Female':1,
    np.nan:10
}
Ever_Married = {
    'Yes':0,
    'No':1,
    np.nan:10
}
Graduated = {
    'Yes':0,
    'No':1,
    np.nan:10
}
Spending_Score = {
    'Low':0,
    'Average':1,
    'High':2,
    np.nan:10
}
to_change = [Gender, Ever_Married, Graduated, Spending_Score, Profession] 

for i in to_change:
  train = train.replace(i)
  test = test.replace(i)

In [14]:
test.isna().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

In [15]:
train.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

In [16]:
train = train.replace({10:np.nan})
test = test.replace({10:np.nan})

In [19]:
from fancyimpute import KNN
train = pd.DataFrame(KNN(k = 3).fit_transform(train), columns = test.columns)
test = pd.DataFrame(KNN(k = 3).fit_transform(test), columns = test.columns)

Using TensorFlow backend.


Imputing row 1/8068 with 0 missing, elapsed time: 10.131
Imputing row 101/8068 with 0 missing, elapsed time: 10.152
Imputing row 201/8068 with 0 missing, elapsed time: 10.156
Imputing row 301/8068 with 1 missing, elapsed time: 10.163
Imputing row 401/8068 with 0 missing, elapsed time: 10.169
Imputing row 501/8068 with 0 missing, elapsed time: 10.175
Imputing row 601/8068 with 0 missing, elapsed time: 10.182
Imputing row 701/8068 with 0 missing, elapsed time: 10.189
Imputing row 801/8068 with 0 missing, elapsed time: 10.195
Imputing row 901/8068 with 0 missing, elapsed time: 10.201
Imputing row 1001/8068 with 0 missing, elapsed time: 10.209
Imputing row 1101/8068 with 0 missing, elapsed time: 10.214
Imputing row 1201/8068 with 0 missing, elapsed time: 10.218
Imputing row 1301/8068 with 0 missing, elapsed time: 10.221
Imputing row 1401/8068 with 1 missing, elapsed time: 10.227
Imputing row 1501/8068 with 0 missing, elapsed time: 10.233
Imputing row 1601/8068 with 1 missing, elapsed time:

In [20]:
for i in train.columns:
  train[i] = train[i].apply(lambda x: int(round(x)))
  test[i] = test[i].apply(lambda x: int(round(x)))

In [21]:
train.dtypes

ID                 int64
Gender             int64
Ever_Married       int64
Age                int64
Graduated          int64
Profession         int64
Work_Experience    int64
Spending_Score     int64
Family_Size        int64
Var_1              int64
dtype: object

In [22]:
y=label
X=train.drop('ID',axis=1)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier

In [26]:
grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
kf = KFold(n_splits=2)

gs = GridSearchCV(estimator = XGBClassifier(n_estimators=500), param_grid = grid, scoring='accuracy',n_jobs=4, cv=kf)

In [27]:
gs.fit(X_train, y_train)

y_pred = gs.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 54.15%


In [29]:
gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=2,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [31]:
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[23:33:37] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 54.34%


In [32]:
X_train.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(
    max_depth=2, 
    random_state=42,
    n_estimators=100,
    
)

clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 48.39%


In [34]:
xgb.fit(X, y)

predictions = xgb.predict(test.drop(['ID'], axis=1))

[23:33:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [35]:
submission = pd.DataFrame({
    'ID':test['ID'],
    'Segmentation':predictions
    })

In [36]:
submission.reset_index(inplace=True, drop=True)

In [37]:
submission.to_csv('final.csv',index=False)